In [ ]:
import os
import import_ipynb
import gmsh
import numpy as np

# Gmsh application programming interface (API)

Gmsh 4.13.1: A three-dimensional finite element mesh generator with built-in pre- and post-processing facilities. https://gmsh.info/

The Gmsh application programming interface (API) allows to integrate the Gmsh library in external applications written in C++, C, Python, Julia or Fortran.

References:  
1. Geometry basics, elementary entities, physical groups. Tutorial t1. https://gmsh.info/doc/texinfo/gmsh.html#t1
2. https://gitlab.onelab.info/gmsh/gmsh/-/tree/gmsh_4_13_1/examples/api
3. https://gitlab.onelab.info/gmsh/gmsh/blob/gmsh_4_13_1/api/gmsh.py
4. https://gmsh.info/doc/texinfo/gmsh.html#Namespace-gmsh_002fmodel_002fgeo

## Namespace gmsh/model/geo: built-in CAD kernel functions

The Python API is entirely defined in the ``gmsh.py`` module (which contains the full documentation of all the functions in the API). 
Before using any functions in the Python API, Gmsh must be initialized: `gmsh.initialize()`.

The first type of 'elementary entity' in Gmsh is a `Point`. A `Point` is uniquely identified by a tag (a strictly positive integer; here `1`) and defined by a list of four numbers: three coordinates (X, Y and Z) and the target mesh size (``lc``) close to the point: Point(1) = {0, 0, 0, lc};

If no target mesh size of provided, a default uniform coarse size will be used for the model, based on the overall model size.

### `gmsh/model/geo/addPoint`   
    
> Add a geometrical point in the built-in CAD representation, at coordinates (x, y, z). If meshSize is > 0, add a meshing constraint at that point. If tag is positive, set the tag explicitly; otherwise a new tag is selected automatically. Return the tag of the point. _(Note that the point will be added in the current model only after synchronize is called. This behavior holds for all the entities added in the geo module.)_

> Input: x (double), y (double), z (double), meshSize = 0. (double), tag = -1 (integer)  
> Output: -  
> Return: integer

In [ ]:
def add_points(points, lc):
    point_tags = []
    for i, (x, y, z) in enumerate(points):
        tag = gmsh.model.geo.addPoint(x, y, z, lc, i + 1)
        point_tags.append(tag)
    return point_tags

Curves are Gmsh's second type of elementery entities, and, amongst curves, straight lines are the simplest. The API to create straight line segments with the built-in kernel follows the same conventions: the first 2 arguments are point tags (the start and end points of the line), and the last (optional one) is the line tag.

### ``gmsh/model/geo/addLine``

> Add a straight line segment in the built-in CAD representation, between the two points with tags startTag and endTag. If tag is positive, set the tag explicitly; otherwise a new tag is selected automatically. Return the tag of the line.

> Input: startTag (integer), endTag (integer), tag = -1 (integer)  
> Output: -  
> Return: integer

In [ ]:
def add_lines(point_tags):
    line_tags = []
    for i in range(len(point_tags)):
        start_point = point_tags[i]
        # a % b retorna o resto da divisão inteira de 'a' por 'b', quando a é maior que b;
        # ou retorna o próprio número 'a' quando ele é menor que 'b', pois a divisão inteira resulta em 0 e todo o valor de a se torna o resto;
        # ou retorna 0 quando 'a' é múltiplo de 'b'.
        end_point = point_tags[(i + 1) % len(point_tags)]  
        tag = gmsh.model.geo.addLine(start_point, end_point)
        line_tags.append(tag)
    return line_tags

The third elementary entity is the surface. In order to define a simple rectangular surface from the four curves defined above, a curve loop has first to be defined. A curve loop is defined by an ordered list of connected curves, a sign being associated with each curve (depending on the orientation of the curve to form a loop). 

### ``gmsh/model/geo/addCurveLoop``

> Add a curve loop (a closed wire) in the built-in CAD representation, formed by the curves curveTags. curveTags should contain (signed) tags of model entities of dimension 1 forming a closed loop: a negative tag signifies that the underlying curve is considered with reversed orientation. If tag is positive, set the tag explicitly; otherwise a new tag is selected automatically. If reorient is set, automatically reorient the curves if necessary. Return the tag of the curve loop.

> Input: curveTags (vector of integers), tag = -1 (integer), reorient = False (boolean)  
> Output: -  
> Return: integer

### ``gmsh/model/geo/addPlaneSurface``

> Add a plane surface in the built-in CAD representation, defined by one or more curve loops wireTags. The first curve loop defines the exterior contour; additional curve loop define holes. If tag is positive, set the tag explicitly; otherwise a new tag is selected automatically. Return the tag of the surface.

> Input: wireTags (vector of integers), tag = -1 (integer)  
> Output: -  
> Return: integer

In [ ]:
def add_surface(line_tags):
    tag_curve = gmsh.model.geo.addCurveLoop(line_tags)
    tag_surface = gmsh.model.geo.addPlaneSurface([tag_curve])
    gmsh.model.geo.synchronize()
    return tag_surface

### `gmsh/model/geo/addCircleArc`

> Add a circle arc (strictly smaller than Pi) in the built-in CAD representation, between the two points with tags ``startTag`` and ``endTag``, and with center ``centerTag``. If ``tag`` is positive, set the tag explicitly; otherwise a new tag is selected automatically. If (``nx``, ``ny``, ``nz``) != (0, 0, 0), explicitly set the plane of the circle arc. Return the tag of the circle arc.

> Input:
> ``startTag`` (integer), ``centerTag`` (integer), ``endTag`` (integer), ``tag`` = -1 (integer), ``nx`` = 0. (double), ``ny`` = 0. (double), ``nz`` = 0. (double)  
> Output: -  
> Return: integer

In [ ]:
def add_circle(center_tag, radius, lc, pts, tag_start):
    factory = gmsh.model.geo
    x, y, z = (0, 0, 0)

    # Criar os pontos no círculo
    point_tags = []
    for i in range(pts):
        angle = i * (2 * np.pi / pts)
        px = x + radius * np.cos(angle)
        py = y + radius * np.sin(angle)
        point_tags.append(factory.addPoint(px, py, z, lc))

    # Criar os arcos do círculo entre os pontos consecutivos
    arc_tags = []
    for i in range(pts):
        start = point_tags[i]
        end = point_tags[(i + 1) % pts]
        arc_tag = factory.addCircleArc(start, center_tag, end, tag_start + i)
        arc_tags.append(arc_tag)

    return arc_tags

## Namespace gmsh/model: model functions

### ``gmsh/model/add``

> Add a new model, with name name, and set it as the current model.

> Input: name (string)  
> Output: -  
> Return: - 

An optional step is needed if we want to group elementary geometrical entities into more meaningful groups, e.g. to define some mathematical ("domain", "boundary"), functional ("left wing", "fuselage") or material ("steel", "carbon") properties. Such groups are called "Physical Groups" in Gmsh.

By default, if physical groups are defined, Gmsh will export in output files only mesh elements that belong to at least one physical group. To force Gmsh to save all elements, whether they belong to physical groups or not, set ``gmsh.option.setNumber("Mesh.SaveAll", 1)``.

Physical groups are also identified by tags, i.e. stricly positive integers, that should be unique per dimension (0D, 1D, 2D or 3D). Physical groups can also be given names.

### ``gmsh/model/addPhysicalGroup``

> Add a physical group of dimension dim, grouping the model entities with tags tags. Return the tag of the physical group, equal to tag if tag is positive, or a new tag if tag < 0. Set the name of the physical group if name is not empty.

> Input: dim (integer), tags (vector of integers), tag = -1 (integer), name = "" (string)  
> Output: -  
> Return: integer

# `rectangular_geometry()`

In [ ]:
def rectangular_geometry(element_type, bc_list, lc, auto_save=True, view_mesh=False):
    type, order = element_type
    vertices = [(0, 0, 0), (1, 0, 0), (1, 1, 0), (0, 1, 0)]

    # Define a new model
    gmsh.initialize()
    gmsh.model.add("rectangular_domain")

    # Define the points of the domain.
    point_tags = add_points(vertices, lc)

    # Criar linhas para formar as bordas do quadrado
    line_tags = add_lines(point_tags)
    
    # Criar um loop de linha e uma superfície plana
    surface_tag = add_surface(line_tags)

    # Adiciona grupos físicos para as condições de contorno
    for bc in bc_list:
        if bc['type'] in ['Dirichlet']:
            gmsh.model.addPhysicalGroup(
                dim=1, tags=line_tags, tag=bc['tag'], name=bc['name'])

    # Atribuindo a tag 202 para o grupo da superfície do domínio
    gmsh.model.addPhysicalGroup(2, [surface_tag], 201, name='entire_domain')  
    
    # Gerar a malha 2D
    gmsh.option.setNumber("Mesh.SaveAll", 1)

    # Definir o tipo de elemento
    if type == 'Quadrangle':
        gmsh.model.mesh.setTransfiniteSurface(surface_tag, "Alternate")
        gmsh.model.mesh.setRecombine(2, surface_tag)

    # Define a ordem dos elementos
    gmsh.model.mesh.generate(2)
    gmsh.model.mesh.setOrder(order)

    # Visualizar a malha no ambiente Gmsh (opcional)
    if view_mesh:
        gmsh.fltk.run()
    
    if auto_save:
        os.makedirs("pre_processing/mesh", exist_ok=True)
        file_path = f"pre_processing/mesh/rectangular_domain_{type}{order}.msh"
        gmsh.write(file_path)
        gmsh.finalize()

# `L_geometry()`

In [ ]:
def L_geometry(element_type, bc_list, lc, auto_save=True, view_mesh=False):
    type, order = element_type

    # Definindo os vértices do domínio em forma de L
    vertices = [(0, 0, 0), (0, -1, 0), (1, -1, 0), (1, 1, 0), (-1, 1, 0), (-1, 0, 0)]

    # Inicializar o Gmsh
    gmsh.initialize()
    gmsh.model.add("L_domain")

    # Adicionar pontos ao modelo
    point_tags = []
    for vertex in vertices:
        # Refinar no ponto (0, 0)
        local_lc = lc * 0.2 if vertex == (0, 0, 0) else lc  
        tag = gmsh.model.geo.addPoint(*vertex, local_lc)
        point_tags.append(tag)

    # Criar as linhas conectando os pontos
    line_tags = [
        gmsh.model.geo.addLine(point_tags[i], point_tags[(i + 1) % len(point_tags)])
        for i in range(len(point_tags))
    ]

    # Criar os loops e as superfícies
    gmsh.model.geo.addCurveLoop(line_tags, 1)
    surface_tag = gmsh.model.geo.addPlaneSurface([1])

    # Adiciona grupos físicos para as condições de contorno
    for bc in bc_list:
        if bc['type'] in ['Dirichlet', 'Neumann']:
            gmsh.model.addPhysicalGroup(
                dim=1, tags=line_tags, tag=bc['tag'], name=bc['name'])

    # Atribuindo a tag 202 para o grupo da superfície do domínio
    gmsh.model.addPhysicalGroup(2, [surface_tag], 201, name='entire_domain') 

    # Sincronizar o modelo
    gmsh.model.geo.synchronize()

    # Configurar o tipo de elemento e recombinar, se necessário
    if type == 'Quadrangle':
        #gmsh.model.mesh.setTransfiniteSurface(surface_tag, "Alternate")
        gmsh.model.mesh.setRecombine(2, surface_tag)

    # Gerar a malha
    gmsh.model.mesh.generate(2)
    gmsh.model.mesh.setOrder(order)

    # Visualizar a malha, se solicitado
    if view_mesh:
        gmsh.fltk.run()

    # Salvar a malha em arquivo
    if auto_save:
        os.makedirs("pre_processing/mesh", exist_ok=True)
        file_path = f"pre_processing/mesh/L_domain_{type}{order}.msh"
        gmsh.write(file_path)
        gmsh.finalize()

# `coaxial_longitudinal()`

In [ ]:
def coaxial_longitudinal(element_type, bc, mp, h, auto_save=True, view_mesh=False):
    # a = 1 cm, b = √3 cm, c = 2 cm
    radii = {'a': 1e-2, 'b': np.sqrt(3) * 1e-2, 'c': 2e-2}  
    type, order = element_type
    
    # Inicializar o Gmsh
    gmsh.initialize()
    try:
        gmsh.model.add("coaxial_longitudinal")
        factory = gmsh.model.occ

        # Criar os loops de curva
        in_boundary = factory.addCircle(0, 0, 0, radii['a'])
        die_boundary = factory.addCircle(0, 0, 0, radii['b'])
        out_boundary = factory.addCircle(0, 0, 0, radii['c'])

        # Criar loops de curva
        in_loop = factory.addCurveLoop([in_boundary])
        die_loop = factory.addCurveLoop([die_boundary])
        out_loop = factory.addCurveLoop([out_boundary])

        # Criar superfícies entre os loops
        inner_conductor = factory.addPlaneSurface([in_loop])  
        dielectric = factory.addPlaneSurface([die_loop, in_loop])  
        outer_conductor = factory.addPlaneSurface([out_loop, die_loop])  

        # Sincronizar geometria
        factory.synchronize()

        # Adicionar grupos físicos para Dim=1
        gmsh.model.addPhysicalGroup(1, [out_boundary], tag=bc[0]['tag'], name=bc[0]['name'])

        # Adicionar grupos físicos para Dim=2
        gmsh.model.addPhysicalGroup(2, [inner_conductor], tag=mp[0]['tag'], name=mp[0]['name'])
        gmsh.model.addPhysicalGroup(2, [dielectric], tag=mp[1]['tag'], name=mp[1]['name'])
        gmsh.model.addPhysicalGroup(2, [outer_conductor], tag=mp[2]['tag'], name=mp[2]['name'])

        # Gerar malha
        print(f"Gerando malha com elementos triangulares de ordem P{order} ...")
        gmsh.option.setNumber("Mesh.SaveAll", 1)
        gmsh.model.mesh.setSize(gmsh.model.getEntities(dim=0), h)
        gmsh.model.mesh.generate(2)
        gmsh.model.mesh.setOrder(order)

        # Salvar e visualizar
        if auto_save:
            os.makedirs("pre_processing/mesh", exist_ok=True)
            file_path = f"pre_processing/mesh/coax_long_domain_{type}{order}.msh"
            gmsh.write(file_path)
            print(f"Malha salva em {file_path}")
        if view_mesh:
            gmsh.fltk.run()
    finally:
        gmsh.finalize()

# `coaxial_quarter_longitudinal()`

In [ ]:
def coaxial_quarter_longitudinal(FINITE_ELEMENT, bc, mp, h, auto_save=True, view_mesh=False):
    # Definição dos raios
    radii = {'a': 1e-2, 'b': np.sqrt(3) * 1e-2, 'c': 2e-2}
    type, order = FINITE_ELEMENT

    # Inicializar o Gmsh
    gmsh.initialize()
    try:
        gmsh.model.add("coaxial_quarter_longitudinal")
        factory = gmsh.model.occ

        # Criar pontos para os arcos no quadrante superior direito
        p0 = factory.addPoint(0, 0, 0)  # Centro do círculo
        p_in_start = factory.addPoint(radii['a'], 0, 0)  # Início do arco interno
        p_in_middle = factory.addPoint(0, radii['a'], 0)  # Fim do arco interno
        p_die_start = factory.addPoint(radii['b'], 0, 0)  # Início do arco dielétrico
        p_die_middle = factory.addPoint(0, radii['b'], 0)  # Fim do arco dielétrico
        p_out_start = factory.addPoint(radii['c'], 0, 0)  # Início do arco externo
        p_out_middle = factory.addPoint(0, radii['c'], 0)  # Fim do arco externo

        # Criar arcos no quadrante superior direito
        arc_in = factory.addCircleArc(p_in_start, p0, p_in_middle)
        arc_die = factory.addCircleArc(p_die_start, p0, p_die_middle)
        arc_out = factory.addCircleArc(p_out_start, p0, p_out_middle)

        # Criar segmentos retos para conectar os arcos
        seg_in_x = factory.addLine(p0, p_in_start)
        seg_die_x = factory.addLine(p_in_start, p_die_start)
        seg_out_x = factory.addLine(p_die_start, p_out_start)
        seg_in_y = factory.addLine(p0, p_in_middle)
        seg_die_y = factory.addLine(p_in_middle, p_die_middle)
        seg_out_y = factory.addLine(p_die_middle, p_out_middle)
        cut_domain = [seg_in_x, seg_die_x, seg_out_x, seg_in_y, seg_die_y, seg_out_y]

        # Criar loops de curva para cada região
        in_loop = factory.addCurveLoop([seg_in_x, arc_in, seg_in_y])
        die_loop = factory.addCurveLoop([seg_die_x, arc_die, seg_die_y, -arc_in])
        out_loop = factory.addCurveLoop([seg_out_x, arc_out, seg_out_y, -arc_die])

        # Criar superfícies preenchidas
        inner_conductor = factory.addPlaneSurface([in_loop])
        dielectric = factory.addPlaneSurface([die_loop])
        outer_conductor = factory.addPlaneSurface([out_loop])

        # Sincronizar geometria
        factory.synchronize()

        # Adicionar grupos físicos para Dim=1 (curvas)
        gmsh.model.addPhysicalGroup(1, [arc_out], tag=bc[0]['tag'], name=bc[0]['name'])
        #gmsh.model.addPhysicalGroup(1, cut_domain, tag=bc[1]['tag'], name=bc[0]['name'])

        # Adicionar grupos físicos para Dim=2 (superfícies)
        gmsh.model.addPhysicalGroup(2, [inner_conductor], tag=mp[0]['tag'], name=mp[0]['name'])
        gmsh.model.addPhysicalGroup(2, [dielectric], tag=mp[1]['tag'], name=mp[1]['name'])
        gmsh.model.addPhysicalGroup(2, [outer_conductor], tag=mp[2]['tag'], name=mp[2]['name'])

        # Gerar malha
        gmsh.option.setNumber("Mesh.SaveAll", 1)
        gmsh.model.mesh.setSize(gmsh.model.getEntities(dim=0), h)
        gmsh.model.mesh.generate(2)
        gmsh.model.mesh.setOrder(order)

        # Salvar e visualizar
        if auto_save:
            os.makedirs("pre_processing/mesh", exist_ok=True)
            file_path = f"pre_processing/mesh/coax_quarter_domain_{type}{order}.msh"
            gmsh.write(file_path)
            print(f"Malha salva em {file_path}")
        if view_mesh:
            gmsh.fltk.run()
    finally:
        gmsh.finalize()

# `axial_rectangular_geometry()`

In [ ]:
def axial_rectangular_geometry(FINITE_ELEMENT, BOUNDARY, MATERIAL, lc, auto_save=True, view_mesh=False):

    # Nome do modelo
    type, order = FINITE_ELEMENT

    # Dimensões do retângulo
    width = 0.5
    height = 0.5
    ra = 0.1
    rb = ra + 0.02205

    # Tamanhos de malha
    min_size = lc * 0.1  # Tamanho mínimo nas linhas refinadas e na casca
    max_size = lc    # Tamanho no restante do domínio

    # Inicializar o Gmsh
    gmsh.initialize()
    try:
        gmsh.model.add("Malha Triangular Refinada Localmente")
        factory = gmsh.model.geo
        
        # Criação dos pontos
        p1 = factory.addPoint(0, 0, 0)  # Inferior esquerdo
        p2 = factory.addPoint(width, 0, 0)  # Inferior direito
        p3 = factory.addPoint(width, height, 0)  # Superior direito
        p4 = factory.addPoint(0, height, 0)  # Superior esquerdo

        # Pontos da casca esférica
        p11 = factory.addPoint(ra, 0, 0)  
        p12 = factory.addPoint(rb, 0, 0)  
        p41 = factory.addPoint(0, rb, 0)  
        p42 = factory.addPoint(0, ra, 0)  

        # Criação dos domínios Neumann Homogêneo Inferior
        l11 = factory.addLine(p1, p11)
        l12 = factory.addLine(p11, p12)
        l13 = factory.addLine(p12, p2)

        # Criação do domínio Dirichlet não-homogêneo
        l2 = factory.addLine(p2, p3)  

        # Superior
        l3 = factory.addLine(p3, p4)

        # Criação do domínio Dirichlet homogêneo
        l41 = factory.addLine(p4, p41)
        l42 = factory.addLine(p41, p42)          
        l43 = factory.addLine(p42, p1)

        # Linhas da casca esférica
        inner_shell = factory.addCircleArc(p11, p1, p42)
        outer_shell = factory.addCircleArc(p12, p1, p41)

        # Criar o contorno da superfície
        air_loop = factory.addCurveLoop([l11, inner_shell, l43])
        magnetic_loop = factory.addCurveLoop([l12, outer_shell, l42, -inner_shell])
        free_loop = factory.addCurveLoop([l13, l2, l3, l41, -outer_shell])

        air_cavity = factory.addPlaneSurface([air_loop])
        magnetic_shell = factory.addPlaneSurface([magnetic_loop])
        free_space = factory.addPlaneSurface([free_loop])

        # Refinamento: aplicar campos de distância para a casca esférica
        gmsh.model.mesh.field.add("Distance", 1)
        gmsh.model.mesh.field.setNumbers(1, "EdgesList", [inner_shell, outer_shell])  # Refinamento nos arcos da casca esférica

        # Refinamento adicional para a linha l41
        gmsh.model.mesh.field.add("Distance", 2)
        gmsh.model.mesh.field.setNumbers(2, "EdgesList", [l41])

        # # Refinamento adicional para a linha l2
        # gmsh.model.mesh.field.add("Distance", 3)
        # gmsh.model.mesh.field.setNumbers(3, "EdgesList", [l2])

        # Refinamento adicional para a linha l11
        gmsh.model.mesh.field.add("Distance", 4)
        gmsh.model.mesh.field.setNumbers(4, "EdgesList", [l11])

        # Refinamento adicional para a linha l43
        gmsh.model.mesh.field.add("Distance", 5)
        gmsh.model.mesh.field.setNumbers(5, "EdgesList", [l43])

        # Combinação de refinamentos
        gmsh.model.mesh.field.add("Min", 6)
        #gmsh.model.mesh.field.setNumbers(6, "FieldsList", [1, 2, 3, 4, 5])
        gmsh.model.mesh.field.setNumbers(6, "FieldsList", [1, 2, 4, 5])

        # Campo Threshold para controlar o refinamento
        gmsh.model.mesh.field.add("Threshold", 7)
        gmsh.model.mesh.field.setNumber(7, "InField", 6)
        gmsh.model.mesh.field.setNumber(7, "SizeMin", min_size)  # Tamanho mínimo nas linhas e casca
        gmsh.model.mesh.field.setNumber(7, "SizeMax", max_size)  # Tamanho máximo fora das regiões refinadas
        gmsh.model.mesh.field.setNumber(7, "DistMin", 0.01)  # Distância mínima para aplicar o tamanho min_size
        gmsh.model.mesh.field.setNumber(7, "DistMax", 0.05)  # Distância máxima para aplicar o tamanho max_size

        # Aplicar o campo de malha combinado
        gmsh.model.mesh.field.setAsBackgroundMesh(7)

        # Definindo as curvas de contorno de Dirichlet (dim=1)
        gmsh.model.addPhysicalGroup(dim=1, tags=[l41, l42, l43], tag=BOUNDARY[0]['tag'], name=BOUNDARY[0]['name'])
        gmsh.model.addPhysicalGroup(dim=1, tags=[l2], tag=BOUNDARY[1]['tag'], name=BOUNDARY[1]['name'])

        # Adicionar grupos físicos para Dim=2 (superfícies)
        gmsh.model.addPhysicalGroup(dim=2, tags=[air_cavity], tag=MATERIAL[0]['tag'], name=MATERIAL[0]['name'])
        gmsh.model.addPhysicalGroup(dim=2, tags=[magnetic_shell], tag=MATERIAL[1]['tag'], name=MATERIAL[1]['name'])
        gmsh.model.addPhysicalGroup(dim=2, tags=[free_space], tag=MATERIAL[2]['tag'], name=MATERIAL[2]['name'])

        # Sincronizar e gerar malha
        factory.synchronize()
        gmsh.model.mesh.generate(2)

        # Configurando a ordem do elemento
        gmsh.model.mesh.setOrder(order)

        # Salvar e visualizar
        if auto_save:
            os.makedirs("pre_processing/mesh", exist_ok=True)
            file_path = f"pre_processing/mesh/axial_rectangular_domain_{type}{order}.msh"
            gmsh.write(file_path)
            print(f"Malha salva em {file_path}")
        if view_mesh:
            gmsh.fltk.run()
    finally:
        gmsh.finalize()

# `magneto_quasi_static_geometry()`

In [ ]:
def magneto_quasi_static_geometry(FINITE_ELEMENT, BOUNDARY, MATERIAL, lc, auto_save=True, view_mesh=False):

    # Nome do modelo
    type, order = FINITE_ELEMENT

    # Dimensões do retângulo
    width = 0.3
    height = 0.3
    ra = 0.05
    rb = 0.055

    # Tamanhos de malha
    min_size = lc * 0.1  # Tamanho mínimo nas linhas refinadas e na casca
    max_size = lc    # Tamanho no restante do domínio

    # Inicializar o Gmsh
    gmsh.initialize()
    try:
        gmsh.model.add("Malha Triangular Refinada Localmente")
        factory = gmsh.model.geo
        
        # Criação dos pontos
        p1 = factory.addPoint(0, 0, 0)  # Inferior esquerdo
        p2 = factory.addPoint(width, 0, 0)  # Inferior direito
        p3 = factory.addPoint(width, height, 0)  # Superior direito
        p4 = factory.addPoint(0, height, 0)  # Superior esquerdo

        # Pontos da casca esférica
        p11 = factory.addPoint(ra, 0, 0)  
        p12 = factory.addPoint(rb, 0, 0)  
        p41 = factory.addPoint(0, rb, 0)  
        p42 = factory.addPoint(0, ra, 0)  

        # Criação dos domínios Neumann Homogêneo Inferior
        l11 = factory.addLine(p1, p11)
        l12 = factory.addLine(p11, p12)
        l13 = factory.addLine(p12, p2)

        # Criação do domínio Dirichlet não-homogêneo
        l2 = factory.addLine(p2, p3)  

        # Superior
        l3 = factory.addLine(p3, p4)

        # Criação do domínio Dirichlet homogêneo
        l41 = factory.addLine(p4, p41)
        l42 = factory.addLine(p41, p42)          
        l43 = factory.addLine(p42, p1)

        # Linhas da casca esférica
        inner_shell = factory.addCircleArc(p11, p1, p42)
        outer_shell = factory.addCircleArc(p12, p1, p41)

        # Criar o contorno da superfície
        air_loop = factory.addCurveLoop([l11, inner_shell, l43])
        magnetic_loop = factory.addCurveLoop([l12, outer_shell, l42, -inner_shell])
        free_loop = factory.addCurveLoop([l13, l2, l3, l41, -outer_shell])

        air_cavity = factory.addPlaneSurface([air_loop])
        magnetic_shell = factory.addPlaneSurface([magnetic_loop])
        free_space = factory.addPlaneSurface([free_loop])

        # Refinamento: aplicar campos de distância para a casca esférica
        gmsh.model.mesh.field.add("Distance", 1)
        gmsh.model.mesh.field.setNumbers(1, "EdgesList", [inner_shell, outer_shell])  # Refinamento nos arcos da casca esférica

        # Refinamento adicional para a linha l41
        gmsh.model.mesh.field.add("Distance", 2)
        gmsh.model.mesh.field.setNumbers(2, "EdgesList", [l41])

        # # Refinamento adicional para a linha l2
        # gmsh.model.mesh.field.add("Distance", 3)
        # gmsh.model.mesh.field.setNumbers(3, "EdgesList", [l2])

        # Refinamento adicional para a linha l11
        gmsh.model.mesh.field.add("Distance", 4)
        gmsh.model.mesh.field.setNumbers(4, "EdgesList", [l11])

        # Refinamento adicional para a linha l43
        gmsh.model.mesh.field.add("Distance", 5)
        gmsh.model.mesh.field.setNumbers(5, "EdgesList", [l43])

        # Combinação de refinamentos
        gmsh.model.mesh.field.add("Min", 6)
        #gmsh.model.mesh.field.setNumbers(6, "FieldsList", [1, 2, 3, 4, 5])
        gmsh.model.mesh.field.setNumbers(6, "FieldsList", [1, 2, 4, 5])

        # Campo Threshold para controlar o refinamento
        gmsh.model.mesh.field.add("Threshold", 7)
        gmsh.model.mesh.field.setNumber(7, "InField", 6)
        gmsh.model.mesh.field.setNumber(7, "SizeMin", min_size)  # Tamanho mínimo nas linhas e casca
        gmsh.model.mesh.field.setNumber(7, "SizeMax", max_size)  # Tamanho máximo fora das regiões refinadas
        gmsh.model.mesh.field.setNumber(7, "DistMin", 0.01)  # Distância mínima para aplicar o tamanho min_size
        gmsh.model.mesh.field.setNumber(7, "DistMax", 0.05)  # Distância máxima para aplicar o tamanho max_size

        # Aplicar o campo de malha combinado
        gmsh.model.mesh.field.setAsBackgroundMesh(7)

        # Definindo as curvas de contorno de Dirichlet (dim=1)
        gmsh.model.addPhysicalGroup(dim=1, tags=[l41, l42, l43], tag=BOUNDARY[0]['tag'], name=BOUNDARY[0]['name'])
        gmsh.model.addPhysicalGroup(dim=1, tags=[l2], tag=BOUNDARY[1]['tag'], name=BOUNDARY[1]['name'])

        # Adicionar grupos físicos para Dim=2 (superfícies)
        gmsh.model.addPhysicalGroup(dim=2, tags=[air_cavity], tag=MATERIAL[0]['tag'], name=MATERIAL[0]['name'])
        gmsh.model.addPhysicalGroup(dim=2, tags=[magnetic_shell], tag=MATERIAL[1]['tag'], name=MATERIAL[1]['name'])
        gmsh.model.addPhysicalGroup(dim=2, tags=[free_space], tag=MATERIAL[2]['tag'], name=MATERIAL[2]['name'])

        # Sincronizar e gerar malha
        factory.synchronize()
        gmsh.model.mesh.generate(2)

        # Configurando a ordem do elemento
        gmsh.model.mesh.setOrder(order)

        # Salvar e visualizar
        if auto_save:
            os.makedirs("pre_processing/mesh", exist_ok=True)
            file_path = f"pre_processing/mesh/magneto_quasi_static_domain_{type}{order}.msh"
            gmsh.write(file_path)
            print(f"Malha salva em {file_path}")
        if view_mesh:
            gmsh.fltk.run()
    finally:
        gmsh.finalize()

# `parallel_plate_capacitor()`

In [ ]:
def parallel_plate_capacitor(FINITE_ELEMENT, BOUNDARY, MATERIAL, lc, auto_save=True, view_mesh=False):

    # Nome do modelo
    type, order = FINITE_ELEMENT

    # Parâmetros geométricos
    Lx, Ly = 5E-2, 5E-2             # Dimensões do domínio truncado
    dy = 1E-2                       # Distâncias entre as placas
    plate_length = 2E-2             # Comprimento das placas
    plate_thickness = 1E-3          # Espessura das placas
    Lc1 = lc                        # Tamanho de malha nas bordas externas
    Lc2 = lc * 0.5                  # Tamanho de malha nos detalhes

    # Coordenadas centrais
    top_plate_y = dy / 2
    bottom_plate_y = -dy / 2

    # Inicializar o Gmsh
    gmsh.initialize()
    try:
        gmsh.model.add("parallel_plate_capacitor")
        # Criar instância da fábrica de geometria
        factory = gmsh.model.geo

        # Definição dos pontos do domínio externo
        p1 = factory.addPoint(-Lx / 2, -Ly / 2, 0, Lc1)
        p2 = factory.addPoint(Lx / 2, -Ly / 2, 0, Lc1)
        p3 = factory.addPoint(Lx / 2, Ly / 2, 0, Lc1)
        p4 = factory.addPoint(-Lx / 2, Ly / 2, 0, Lc1)

        # Definição dos pontos das placas
        # Placa superior
        p5 = factory.addPoint(-plate_length / 2, top_plate_y - plate_thickness / 2, 0, Lc2)
        p6 = factory.addPoint(plate_length / 2, top_plate_y - plate_thickness / 2, 0, Lc2)
        p7 = factory.addPoint(plate_length / 2, top_plate_y + plate_thickness / 2, 0, Lc2)
        p8 = factory.addPoint(-plate_length / 2, top_plate_y + plate_thickness / 2, 0, Lc2)

        # Placa inferior
        p9 = factory.addPoint(-plate_length / 2, bottom_plate_y - plate_thickness / 2, 0, Lc2)
        p10 = factory.addPoint(plate_length / 2, bottom_plate_y - plate_thickness / 2, 0, Lc2)
        p11 = factory.addPoint(plate_length / 2, bottom_plate_y + plate_thickness / 2, 0, Lc2)
        p12 = factory.addPoint(-plate_length / 2, bottom_plate_y + plate_thickness / 2, 0, Lc2)

        # Linhas do domínio externo
        l1 = factory.addLine(p1, p2)
        l2 = factory.addLine(p2, p3)
        l3 = factory.addLine(p3, p4)
        l4 = factory.addLine(p4, p1)

        # Linhas das placas (superior)
        l5 = factory.addLine(p5, p6)
        l6 = factory.addLine(p6, p7)
        l7 = factory.addLine(p7, p8)
        l8 = factory.addLine(p8, p5)

        # Linhas das placas (inferior)
        l9 = factory.addLine(p9, p10)
        l10 = factory.addLine(p10, p11)
        l11 = factory.addLine(p11, p12)
        l12 = factory.addLine(p12, p9)

        # Linhas para o dielétrico (entre as placas)
        l13 = factory.addLine(p5, p12)
        l14 = factory.addLine(p6, p11)

        # Criar loops para os buracos (áreas das placas)
        top_plate = [l5, l6, l7, l8]
        bottom_plate = [l9, l10, l11, l12]
        loop_top_plate = factory.addCurveLoop(top_plate)
        loop_bottom_plate = factory.addCurveLoop(bottom_plate)

        # Criar o loop para o meio dielétrico (entre as placas)
        dielectric = [-l11, -l14, -l5, l13]
        loop_dielectric = factory.addCurveLoop(dielectric)

        # Criar o loop para o domínio externo
        air_domain = [l1, l2, l3, l4]
        loop_air_domain = factory.addCurveLoop(air_domain)

        # Criar a superfície do domínio externo com buracos para as placas
        surface_free_space = factory.addPlaneSurface([loop_air_domain, loop_top_plate, loop_bottom_plate, loop_dielectric])
        surface_dielectric = factory.addPlaneSurface([loop_dielectric])    

        # Definindo as curvas de contorno de Dirichlet (dim=1)
        curves_list = [air_domain, top_plate, bottom_plate]
        for i, contour in enumerate(curves_list):
            gmsh.model.addPhysicalGroup(dim=1, tags=contour, tag=BOUNDARY[i]['tag'], name=BOUNDARY[i]['name'])

        # Adicionar grupos físicos para Dim=2 (superfícies)
        surfaces_list = [surface_free_space, surface_dielectric]
        for i, surface in enumerate(surfaces_list):
            gmsh.model.addPhysicalGroup(dim=2, tags=[surface], tag=MATERIAL[i]['tag'], name=MATERIAL[i]['name'])

        # Aplicar cores para visualização
        gmsh.model.setColor([(2, surface_free_space)], 127, 127, 255)  # Azul claro
        gmsh.model.setColor([(2, surface_dielectric)], 255, 127, 127)  # Vermelho claro

        # Sincronizar e gerar malha
        factory.synchronize()
        gmsh.model.mesh.generate(2)

        # Configurando a ordem do elemento
        gmsh.model.mesh.setOrder(order)

        # Salvar e visualizar
        if auto_save:
            os.makedirs("pre_processing/mesh", exist_ok=True)
            file_path = f"pre_processing/mesh/parallel_plate_capacitor_domain_{type}{order}.msh"
            gmsh.write(file_path)
            print(f"Malha salva em {file_path}")
        if view_mesh:
            gmsh.fltk.run()
    finally:
        gmsh.finalize()

Conversão do arquivo Jupyter Notebook para um script Python: ``python -m nbconvert --to script name.ipynb``

Belo Horizonte, Brazil.  
Adilton Junio Ladeira Pereira - adt@ufmg.br  
&copy; All rights reserved.

version 1.0. November, 2024.